In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import panel as pn

pn.extension("ipywidgets", "katex", "mathjax")

import sys
from inspect import signature
from random import shuffle, uniform

import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.figure import Figure

print("Packages successfully loaded")

# Functions from cookbook

In [ ]:
try:
    %run Initialize/init_cookbook.ipynb # valid when running the cookbook in the main file
except:
    %run init_cookbook.ipynb # valid when running the cookbook from this file.

## New functions for cookbook

In [ ]:
def check_code_function_RANGE(
    fig,
    function_name,
    correct_function,
    f_margin=0.001,
    xlabel=None,
    ylabel=None,
    index_answer="",
    new_graph=True,
    ax=None,
    pane=None,
    show_legend=True,
):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    if new_graph == True:
        pane = pn.pane.Matplotlib(fig, dpi=100)
        ax = fig.subplots()

    if xlabel != None:
        ax.set_xlabel(xlabel)
        fig.subplots_adjust(bottom=0.25)

    if ylabel != None:
        ax.set_ylabel(ylabel)
        fig.subplots_adjust(left=0.15)

    #  Plot the answer if the student function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/
        sig = signature(function)
        student_function = function_name + str(sig)
        student_answer = eval(student_function)
        # student_answer = eval(student_function)[0]

        sig = signature(correct_function)
        correct_function2 = str(correct_function.__name__) + str(sig)
        # correct_answer = eval(correct_function2)

        if index_answer == "":
            student_answer = eval(student_function)
            correct_answer = eval(correct_function2)
        else:
            student_answer = eval(student_function)[index_answer]
            correct_answer = eval(correct_function2)[index_answer]

        # plot the answers
        line = ax.plot(student_answer, label="Your answer", zorder=1)
        line = ax.plot(correct_answer, label="Correct answer", zorder=1)

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        student_answer[student_answer == 0] = 1e-100  # avoid dividing by zero
        inaccuracy = np.abs(1 - np.array(correct_answer) / np.array(student_answer))

        # Plot comment if the answer is correct
        y_loc = (np.nanmean(correct_answer) + np.nanmin(correct_answer)) / 2
        x_loc = np.mean(ax.get_xticks())
        if np.nanmax(changes) == 0:
            text = ax.text(
                x_loc,
                y_loc,
                "Perfect!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
                zorder=0,
            )

        if np.nanmax(changes) != 0 and np.nanmax(inaccuracy) < f_margin:
            text = ax.text(
                x_loc,
                y_loc,
                "Good!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
                zorder=0,
            )

        # show legend,
        if show_legend == True:
            ax.legend()

        # set title
        title = student_function
        title = title.replace("_", " ")
        ax.set_title(title)

    except:
        text_failed = "Almost there, \n your function can not be plotted, \n please try to fix the bug"
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(
            np.average(x_ticks),
            np.average(y_ticks),
            text_failed,
            fontsize=16,
            color="r",
            ha="center",
            va="center",
        )

    # update the graph
    if new_graph == True:
        display(pane)
    else:
        return pane, ax

# Commonly used functions

In [ ]:
def wave_length(T, h):
    d = h

    # based on waveNumber_Fenton(T,d) from Jaime in computerlab
    g = 9.81
    omega = 2 * np.pi / T
    k0 = omega * omega / g
    alpha = k0 * d
    beta = alpha * (np.tanh(alpha)) ** -0.5
    k = (
        (alpha + beta**2 * np.cosh(beta) ** -2)
        / (np.tanh(beta) + beta * np.cosh(beta) ** -2)
        / d
    )

    L = 2 * np.pi / k

    return L


def group_stats(k1, k2, w1, w2):
    Delta_k = np.abs(k2 - k1)
    Delta_w = np.abs(w2 - w1)
    L = 2 * np.pi / Delta_k
    T = 2 * np.pi / Delta_w
    cg = Delta_w / Delta_k
    return L, T, cg

# Normal incident waves

In [ ]:
# H0 = 2.0                        # The offshore wave height [m]
# T = 6                            # The wave period [s]
# h0 = 20                          # The largest water depth
# slope = 1/100                    # The constant bed slope
# rho = 1025                       # The density of water [kg/m3]
# x_range = np.arange(0,2000+1,1)  # The cross-shore directed grid, the horizontal axis. (stepssize of 1 is required to function properly)


def W3_normal_wave(x_range, H0, T, h0, slope):
    # The environmental conditions
    x = x_range  # the horizontal axis
    zbed = -h0 + slope * x  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[
        h < 0
    ] = 0  # no negative depths (A depth of 0 can cause warnings, these can be ignored to keep the code straightforward)

    # given conditions
    gamma = 0.8  # wave breaking ratio

    """ source: Computerlab by Jaime """
    # The wave characteristics at every location in the cross-section
    L = np.array([wave_length(T, h) for h in h])  # The wavelength
    c = L / T  # The wave celerity
    k = 2 * np.pi / L  # The wave number
    n = 0.5 + (k * h / np.sinh(2 * k * h))
    cg = n * c  # The wave group celerity
    Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

    H = H0 * Ksh  # The wave height due to shoaling and refraction
    Hbreaking = gamma * h  # The wave-breaking height
    H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
    g = 9.81
    E = 1 / 8 * rho * g * H**2  # The wave energy

    Sxx = (2 * n - 0.5) * E  # Radiant stresses

    eta = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
    for i in range(len(eta) - 1):  # key here is that setup[0] = 0
        eta[i + 1] = eta[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

    return H, n, Ksh, E, Sxx, eta

In [ ]:
def W3_plot_normal_incident_waves():
    # define the name of the function that the students will make
    function_name = "W3_normal_wave"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -h0 + slope * x  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[
            h < 0
        ] = 0  # no negative depths (A depth of 0 can cause warnings, these can be ignored to keep the code straightforward)

        # given conditions
        gamma = 0.8  # wave breaking ratio

        """ source: Computerlab by Jaime """
        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        H = H0 * Ksh  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Sxx = (2 * n - 0.5) * E  # Radiant stresses

        eta = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
        for i in range(len(eta) - 1):  # key here is that setup[0] = 0
            eta[i + 1] = eta[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

        return H, n, Ksh, E, Sxx, eta

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig, axs = plt.subplots(
        nrows=3, ncols=2, figsize=(10, 4), sharex=True, sharey=False
    )
    pane = pn.pane.Matplotlib(fig, dpi=100)

    # call the function that builds the backend.
    new_graph = False
    ax = axs[0, 0]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="H [m]",
        index_answer=0,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.legend(bbox_to_anchor=(-0.02, 1), loc="lower left")
    ax.set_title("")

    ax = axs[1, 0]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="n",
        index_answer=1,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")

    ax = axs[2, 0]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="Ksh",
        index_answer=2,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")

    ax = axs[0, 1]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="E $[m^2]$",
        index_answer=3,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")

    ax = axs[1, 1]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="Sxx [N/m]",
        index_answer=4,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")

    ax = axs[2, 1]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="$\eta$ [m]",
        index_answer=5,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")


# W3_plot_normal_incident_waves()

## Interactive graph

In [ ]:
def W3_interactiveplot_normal_indicent_waves(H0, T, h0, slope_in):
    slope = 1.0 / slope_in  # bed slope [-]
    h0  # offshore water depth [m]
    x_max = round((h0 + 2) / slope)
    x = np.linspace(0, x_max, 500)  # cross-shore coordinate [m]
    zbed = -(h0 - slope * x)  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    # given:
    gamma = 0.8  # wave breaking ratio
    rho = 1025  # density of (the) water [kg/m3]

    # The wave characteristics at every location in the cross-section
    L = np.array([wave_length(T, h) for h in h])  # The wavelength
    c = L / T  # The wave celerity
    k = 2 * np.pi / L  # The wave number
    n = 0.5 + (k * h / np.sinh(2 * k * h))
    cg = n * c  # The wave group celerity
    Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

    H = H0 * Ksh  # The wave height due to shoaling (only)
    H_shoal = H.copy()  # copy, only for making a graph
    Hbreaking = gamma * h  # The wave-breaking height
    H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # The wave height

    g = 9.81
    E = 1 / 8 * rho * g * H**2  # The wave energy
    Sxx = (2 * n - 0.5) * E  # Radiant stresses

    """ Adjusted the code to implement oblique waves, source: Computerlab by Jaime """
    setup = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
    for i in range(len(setup) - 1):  # key here is that setup[0] = 0
        setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

    fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0, 0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0, 0].plot(
        [0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]"
    )
    axs[1, 0].plot(x_water, L, label="Wavelength (L) [m]", color="k")
    axs[2, 0].plot(x_water, k, label="Wave number (k) [rad/m]", color="k")
    axs[3, 0].plot(x_water, c, label="Wave celerity (c) [m]", color="k")
    axs[4, 0].plot(x_water, cg, label="Wavegroup celerity (cg) [m/s]", color="k")
    axs[0, 1].plot(x_water, Ksh, label="Shoaling factor (ksh) [-]", color="k")
    axs[1, 1].plot(x_water, H_shoal, label="Wave height (only shoaling) [m]", color="k")
    axs[2, 1].plot(x_water, H, label="(Breaking) Wave height [m]", color="k")
    axs[3, 1].plot(x_water, Sxx, label="Radiant stress (Sxx) [N/m]", color="k")
    axs[4, 1].plot(x_water, setup, label="Water level ($\eta$) [m]", color="k")

    # set legends etc for all subplots
    for ax in axs:
        ax[0].legend(loc="lower left")
        ax[0].xaxis.set_visible(False)
        ax[1].legend(loc="best")
        ax[1].xaxis.set_visible(False)

    # show x-axis for the lowest plots
    axs[4, 0].xaxis.set_visible(True)
    axs[4, 1].xaxis.set_visible(True)

    # describe x-axis and make space for label
    fig.subplots_adjust(wspace=0.15)
    axs[4, 0].set_xlabel("Cross-shore location (x) [m]")
    axs[4, 1].set_xlabel("Cross-shore location (x) [m]")


def W3_plot_normal_indicent_waves():
    # Create interactive widgets, which require IPY Widgets, widgets from panel do not work
    # H0 = pn.widgets.FloatInput(value=1.5, start=0, end=500, step=0.1, name="Offshore wave height (H0) [m]")
    # T = pn.widgets.FloatInput(value=5, start=0.05, end=500, step=0.01, name ="Wave period (T) [s]")
    # slope = pn.widgets.FloatInput(value=30, start=0.1, end=50, step=0.1, name ="slope 1:...")
    # h0 = pn.widgets.FloatInput(value=50, start=0.1, end =500, step=0.1, name="offshore depth (h0) [m]")

    H0 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    slope = ipw.FloatText(
        value=100, min=0.1, max=250, step=0.1, description="slope 1:..."
    )
    h0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox([H0, T])
    Vbox2 = ipw.VBox([h0, slope])

    widgets = ipw.HBox([Vbox1, Vbox2])
    graph = ipw.interactive_output(
        W3_interactiveplot_normal_indicent_waves,
        {"H0": H0, "T": T, "h0": h0, "slope_in": slope},
    )

    display(widgets, graph)


# W3_plot_normal_indicent_waves()


def W3_Q1():
    W3_plot_normal_indicent_waves()

    # Q1_text = 'Which method is used to calculate the wavelength?'
    # Q1_cor = ['Iterative approach']
    # Q1_false = ['Linear interpolating tables', 'An explicit formula']

    # Q2_text = 'Which increases the width of the shoaling zone?'
    # Q2_cor = ['Increasing wave period (T)', 'Decreasing bed slope']
    # Q2_false = ['Increasing offshore wave height (H0)', 'Increasing deep water depth']

    Q3_text = "Which increases the wave breaking height?"
    Q3_cor = [" A larger offshore wave height (H0)", " A larger wave period (T)"]
    Q3_false = ["A steeper bed slope", "Increasing the deep water depth"]

    # Q4_text = 'Which influences the set-down?'
    # Q4_cor = ['The wave period (T)']
    # Q4_false = ['The deepwater wave height (H0)', 'The bed slope', 'The deep water depth']

    Q4_text = "Which of the following changes in parameters will increase the width of the surf zone"
    Q4_cor = [
        "Increasing deepwater wave height (H0)",
        "Increasing the wave period (T)",
        "Decreasing the bed slope",
    ]
    Q4_false = ["Decreasing the deep water depth"]

    Q_texts = [Q3_text, Q4_text]
    Q_cor_all = [Q3_cor, Q4_cor]
    Q_false_all = [Q3_false, Q4_false]

    for Q_text, Q_cor, Q_false in zip(Q_texts, Q_cor_all, Q_false_all):
        display(multiple_selection(Q_text, Q_cor, Q_false))

    # disabled, now uses a list rather than eval()
    # n_questions = 4
    # for i in np.arange(1,n_questions+1,1):
    #    Q = eval('Q' + str(i) + '_text')
    #    C = eval('Q' + str(i) + '_cor')
    #    F = eval('Q' + str(i) + '_false')
    #    display(multiple_selection(Q,C,F))


# W3_Q1()

# Oblique waves

### H, c, Sxx, eta

#### Coding question

In [ ]:
def W3_plot_oblique_waves():
    # define the name of the function that the students will make
    function_name = "W3_wave_setup"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope, angle):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(h0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy
        Sxx = (
            n - 0.5 + n * np.cos(theta_radians) ** 2
        ) * E  # radiant stresses for oblique waves, for normal incident waves holds: Sxx = (2*n-0.5)*E

        """ Adjusted the code to implement oblique waves, source: Computerlab by Jaime """
        setup = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
        for i in range(len(setup) - 1):  # key here is that setup[0] = 0
            setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

        return H, Sxx, setup

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(6, 4), sharex=True, sharey=False)
    pane = pn.pane.Matplotlib(fig, dpi=100)

    # call the function that builds the backend.
    new_graph = False
    ax = axs[0]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="H [m]",
        index_answer=0,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.legend(bbox_to_anchor=(-0.02, 1), loc="lower left")
    ax.set_title("")

    ax = axs[1]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="Sxx [N/m]",
        index_answer=1,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")

    ax = axs[2]
    pane, ax = check_code_function_RANGE(
        fig,
        function_name,
        correct_function,
        f_margin=0.01,
        xlabel=None,
        ylabel="$\eta$ [m]",
        index_answer=2,
        new_graph=new_graph,
        ax=ax,
        pane=pane,
        show_legend=False,
    )
    ax.set_title("")


# W3_plot_oblique_waves()

#### Interactive graph

In [ ]:
def W3_interactiveplot_oblique_waves(H0_1, H0_2, T1, T2, angle1, angle2, h0, slope_in):
    slope = 1.0 / slope_in  # bed slope [-]
    h0  # offshore water depth [m]
    x_max = round((h0 + 2) / slope)
    x = np.linspace(0, x_max, 500)  # cross-shore coordinate [m]
    zbed = -h0 + slope * x  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]")

    def calc(T, H0, angle, nr):
        # given:
        gamma = 0.8  # wave breaking ratio
        rho = 1025

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy
        Sxx = (
            n - 0.5 + n * np.cos(theta_radians) ** 2
        ) * E  # radiant stresses for oblique waves, for normal incident waves holds: Sxx = (2*n-0.5)*E

        """ Adjusted the code to implement oblique waves, source: Computerlab by Jaime """
        setup = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
        for i in range(len(setup) - 1):  # key here is that setup[0] = 0
            setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

        axs[1].plot(x_water, H, label="H$_" + str(nr) + "$")
        axs[2].plot(x_water, Sxx, label="Sxx$_" + str(nr) + "$")
        axs[3].plot(x_water, setup, label="Setup$_" + str(nr) + "$")

    # Calculate and plot the lines
    calc(T1, H0_1, angle1, nr=1)
    calc(T2, H0_2, angle2, nr=2)

    # adjust the layout
    for ax in axs:
        ax.legend(loc="lower left")
        ax.xaxis.set_visible(False)
        ax.legend(loc="best")
        ax.xaxis.set_visible(False)
    axs[3].xaxis.set_visible(True)

    # add descriptions
    axs[0].set_ylabel("Cross section")
    axs[1].set_ylabel("Wave height \n (H) [m]")
    axs[2].set_ylabel("Radiation stress \n (Sxx) [N/m]")
    axs[3].set_ylabel("Water level \n (eta) [m]")
    axs[3].set_xlabel("Cross-shore location (x) [m]")


def W3_show_oblique_waves():
    H0_1 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    H0_2 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T1 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")

    angle1 = ipw.FloatText(
        value=0, min=0, max=90, step=0.1, description="angle 1 [O]"
    )  # degree
    angle2 = ipw.FloatText(value=45, min=0, max=90, step=0.1, description="angle 2 [O]")
    slope = ipw.FloatText(
        value=100, min=0.1, max=250, step=0.1, description="slope 1:..."
    )
    h0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), H0_1, T1, angle1]
    )
    Vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), H0_2, T2, angle2]
    )
    Vbox3 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), h0, slope]
    )

    widgets = ipw.HBox([Vbox1, Vbox2, Vbox3])
    graph = ipw.interactive_output(
        W3_interactiveplot_oblique_waves,
        {
            "H0_1": H0_1,
            "H0_2": H0_2,
            "T1": T1,
            "T2": T2,
            "angle1": angle1,
            "angle2": angle2,
            "h0": h0,
            "slope_in": slope,
        },
    )

    display(widgets, graph)


# W3_show_oblique_waves()

### Fx

#### Coding question

In [ ]:
def W3_plot_Fx():
    # define the name of the function that the students will make
    function_name = "W3_Fx"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope, angle):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(h0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        # Sxx = (2*n-0.5)*E                             # Radiant stresses for oblique waves
        Sxx = (n - 0.5 + n * np.cos(theta_radians) ** 2) * E  # Radiant stresses

        setup = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
        for i in range(len(setup) - 1):  # key here is that setup[0] = 0
            setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

        Fx = np.zeros(setup.shape)
        for i in range(len(Fx) - 1):
            Fx[i] = -(Sxx[i + 1] - Sxx[i]) / (x[i + 1] - x[i])

        return Fx

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig = Figure((5, 2.5))
    xlabel = "Cross-shore location (x) [m]"
    ylabel = "Fx [$N/m^2$]"

    # call the function that builds the backend.
    check_code_function_RANGE(
        fig, function_name, correct_function, f_margin, xlabel=xlabel, ylabel=ylabel
    )

#### Graph

In [ ]:
def W3_interactiveplot_Fx(H0_1, H0_2, T1, T2, angle1, angle2, h0, slope_in):
    slope = 1.0 / slope_in  # bed slope [-]
    h0  # offshore water depth [m]
    x_max = round((h0 + 2) / slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    zbed = -h0 + slope * x  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]")

    def calc(T, H0, angle, nr):
        # given:
        gamma = 0.8  # wave breaking ratio
        rho = 1025

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy
        Sxx = (
            n - 0.5 + n * np.cos(theta_radians) ** 2
        ) * E  # radiant stresses for oblique waves, for normal incident waves holds: Sxx = (2*n-0.5)*E

        Fx = np.zeros(Sxx.shape)
        for i in range(len(Fx) - 1):
            Fx[i] = -(Sxx[i + 1] - Sxx[i]) / (x[i + 1] - x[i])

        axs[1].plot(x_water, H, label="H$_" + str(nr) + "$")
        axs[2].plot(x_water, Sxx, label="Sxx$_" + str(nr) + "$")
        axs[3].plot(x_water, Fx, label="Fx$_" + str(nr) + "$")

    # Calculate and plot the lines
    calc(T1, H0_1, angle1, nr=1)
    calc(T2, H0_2, angle2, nr=2)

    # adjust the layout
    for ax in axs:
        ax.legend(loc="lower left")
        ax.xaxis.set_visible(False)
        ax.legend(loc="best")
        ax.xaxis.set_visible(False)
    axs[3].xaxis.set_visible(True)

    # add descriptions
    axs[0].set_ylabel("Cross section")
    axs[1].set_ylabel("Wave height \n (H) [m]")
    axs[2].set_ylabel("Radiation stress \n (Sxx) [N/m]")
    axs[3].set_ylabel("Cross-shore force \n (Fx) [N]")
    axs[3].set_xlabel("Cross-shore location (x) [m]")


def W3_show_Fx():
    H0_1 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    H0_2 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T1 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")

    angle1 = ipw.FloatText(
        value=0, min=0, max=90, step=0.1, description="angle 1 [O]"
    )  # degree
    angle2 = ipw.FloatText(value=45, min=0, max=90, step=0.1, description="angle 2 [O]")
    slope = ipw.FloatText(
        value=100, min=0.1, max=250, step=0.1, description="slope 1:..."
    )
    h0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), H0_1, T1, angle1]
    )
    Vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), H0_2, T2, angle2]
    )
    Vbox3 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), h0, slope]
    )

    widgets = ipw.HBox([Vbox1, Vbox2, Vbox3])
    graph = ipw.interactive_output(
        W3_interactiveplot_Fx,
        {
            "H0_1": H0_1,
            "H0_2": H0_2,
            "T1": T1,
            "T2": T2,
            "angle1": angle1,
            "angle2": angle2,
            "h0": h0,
            "slope_in": slope,
        },
    )

    display(widgets, graph)


# W3_show_Fx()

### Syx + Fy

#### Coding question Syx

In [ ]:
def W3_plot_Syx():
    # define the name of the function that the students will make
    function_name = "W3_Syx"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope, angle):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(h0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        return Syx

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig = Figure((5, 2.5))
    xlabel = "Cross-shore location (x) [m]"
    ylabel = "Syx [N/m]"

    # call the function that builds the backend.
    check_code_function_RANGE(
        fig, function_name, correct_function, f_margin, xlabel=xlabel, ylabel=ylabel
    )

#### Coding question Fy

In [ ]:
def W3_plot_Fy():
    # define the name of the function that the students will make
    function_name = "W3_Fy"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope, angle):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(h0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        Fy = np.zeros(Syx.shape)
        for i in range(len(Fy) - 1):
            Fy[i] = -(Syx[i + 1] - Syx[i]) / (x[i + 1] - x[i])

        return Fy

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig = Figure((5, 2.5))
    xlabel = "Cross-shore location (x) [m]"
    ylabel = "Fy [$N/m^2$]"

    # call the function that builds the backend.
    check_code_function_RANGE(
        fig, function_name, correct_function, f_margin, xlabel=xlabel, ylabel=ylabel
    )

#### Graph

In [ ]:
def W3_interactiveplot_Syx_Fy(H0_1, H0_2, T1, T2, angle1, angle2, h0, slope_in):
    slope = 1.0 / slope_in  # bed slope [-]
    h0  # offshore water depth [m]
    x_max = round((h0 + 2) / slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    zbed = -h0 + slope * x  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]")

    def calc(T, H0, angle, nr):
        # given:
        gamma = 0.8  # wave breaking ratio
        rho = 1025

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E  # alongshore stress

        Fy = np.zeros(Syx.shape)  # alongshore force
        for i in range(len(Fy) - 1):
            Fy[i] = -(Syx[i + 1] - Syx[i]) / (x[i + 1] - x[i])

        axs[1].plot(x_water, H, label="H$_" + str(nr) + "$")
        axs[2].plot(x_water, Syx, label="Syx$_" + str(nr) + "$")
        axs[3].plot(x_water, Fy, label="Fy$_" + str(nr) + "$")

    # Calculate and plot the lines
    calc(T1, H0_1, angle1, nr=1)
    calc(T2, H0_2, angle2, nr=2)

    # adjust the layout
    for ax in axs:
        ax.legend(loc="lower left")
        ax.xaxis.set_visible(False)
        ax.legend(loc="best")
        ax.xaxis.set_visible(False)
    axs[3].xaxis.set_visible(True)

    # add descriptions
    axs[0].set_ylabel("Cross section")
    axs[1].set_ylabel("Wave height \n (H) [m]")
    axs[2].set_ylabel("Syx [N/m]")
    axs[3].set_ylabel("Fy [$N/m^2$]")
    axs[3].set_xlabel("Cross-shore location (x) [m]")


def W3_show_Syx_Fy():
    H0_1 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    H0_2 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T1 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")

    angle1 = ipw.FloatText(
        value=0, min=0, max=90, step=0.1, description="angle 1 [O]"
    )  # degree
    angle2 = ipw.FloatText(value=45, min=0, max=90, step=0.1, description="angle 2 [O]")
    slope = ipw.FloatText(
        value=30, min=0.1, max=50, step=0.1, description="slope 1:..."
    )
    h0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), H0_1, T1, angle1]
    )
    Vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), H0_2, T2, angle2]
    )
    Vbox3 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), h0, slope]
    )

    widgets = ipw.HBox([Vbox1, Vbox2, Vbox3])
    graph = ipw.interactive_output(
        W3_interactiveplot_Syx_Fy,
        {
            "H0_1": H0_1,
            "H0_2": H0_2,
            "T1": T1,
            "T2": T2,
            "angle1": angle1,
            "angle2": angle2,
            "h0": h0,
            "slope_in": slope,
        },
    )

    display(widgets, graph)


# W3_show_Syx_Fy()

### V

#### coding

In [ ]:
def W3_plot_V():
    # define the name of the function that the students will make
    function_name = "W3_V"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope, angle, cf):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(h0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        dh_dx = np.zeros(len(h))  # dh/dx
        for i in range(len(dh_dx) - 1):
            dh_dx[i] = (h[i + 1] - h[i]) / (x[i + 1] - x[i])

        # page 198
        # r = 0.06                                  # bottom roughness [m]
        # C = 18*np.log(12*h/r)                     # Chezy coeffcient
        # c_f = g/C**2                              # friction factor
        # c_f = 0.01

        # formula 5.82 on page 226 of the book
        V = (
            -5
            / 16
            * np.pi
            * gamma
            / cf
            * g
            * np.sin(theta_radians[0])
            / c[0]
            * h
            * dh_dx
        )
        V[H != Hbreaking] = 0  # function valid where waves are breaking

        # formula 5.83 on page 226 of the book
        # Hb = H[H == Hbreaking][0]
        # tan_alpha = dh_dx[i]
        # hb = gamma * Hb

        # V2 = -5/16*np.pi*Hb/c_f*g*np.sin(theta_radians[0])/c[0]*h/hb * tan_alpha
        # V2[H != Hbreaking] = 0

        # including the impact of surface elevation increase
        # h = h + eta
        # V3 = -5/16*np.pi*Hb/c_f*g*np.sin(theta_radians[0])/c[0]*h/hb * tan_alpha
        # V3[H != Hbreaking] = 0

        # print(f'Hb {Hb}, tan alpha {tan_alpha}, hb {hb}, eta {eta[-1]}')

        return V

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig = Figure((5, 2.5))
    xlabel = "Cross-shore location (x) [m]"
    ylabel = "V [m/s]"

    # call the function that builds the backend.
    check_code_function_RANGE(
        fig, function_name, correct_function, f_margin, xlabel=xlabel, ylabel=ylabel
    )

#### Graph

In [ ]:
def W3_interactiveplot_V(H0_1, H0_2, T1, T2, angle1, angle2, h0, slope_in):
    slope = 1.0 / slope_in  # bed slope [-]
    h0  # offshore water depth [m]
    x_max = round((h0 + 2) / slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    zbed = -h0 + slope * x  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]")

    def calc(T, H0, angle, nr):
        # given:
        gamma = 0.8  # wave breaking ratio
        rho = 1025

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        dh_dx = np.zeros(len(h))  # dh/dx
        for i in range(len(dh_dx) - 1):
            dh_dx[i] = (h[i + 1] - h[i]) / (x[i + 1] - x[i])

        # page 198, no wave effect included
        # r = 0.06                                  # bottom roughness [m]
        # C = 18*np.log(12)*h/r                     # Chezy coeffcient
        # c_f = g/C**2                              # friction factor
        c_f = 0.01

        # formula 5.82 on page 226 of the book
        V = (
            -5
            / 16
            * np.pi
            * gamma
            / c_f
            * g
            * np.sin(theta_radians[0])
            / c[0]
            * h
            * dh_dx
        )
        V[H != Hbreaking] = 0

        axs[1].plot(x_water, H, label="H$_" + str(nr) + "$")
        axs[2].plot(x_water, Syx, label="Syx$_" + str(nr) + "$")
        axs[3].plot(x_water, V, label="V$_" + str(nr) + "$")

    # Calculate and plot the lines
    calc(T1, H0_1, angle1, nr=1)
    calc(T2, H0_2, angle2, nr=2)

    # adjust the layout
    for ax in axs:
        ax.legend(loc="lower left")
        ax.xaxis.set_visible(False)
        ax.legend(loc="best")
        ax.xaxis.set_visible(False)
    axs[3].xaxis.set_visible(True)

    # add descriptions
    axs[0].set_ylabel("Cross section")
    axs[1].set_ylabel("Wave height \n (H) [m]")
    axs[2].set_ylabel("Syx [N/m]")
    axs[3].set_ylabel("V [m/s]")
    axs[3].set_xlabel("Cross-shore location (x) [m]")


def W3_show_V():
    H0_1 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    H0_2 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T1 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")

    angle1 = ipw.FloatText(
        value=0, min=0, max=90, step=0.1, description="angle 1 [O]"
    )  # degree
    angle2 = ipw.FloatText(value=45, min=0, max=90, step=0.1, description="angle 2 [O]")
    slope = ipw.FloatText(
        value=100, min=0.1, max=250, step=0.1, description="slope 1:..."
    )
    h0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), H0_1, T1, angle1]
    )
    Vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), H0_2, T2, angle2]
    )
    Vbox3 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), h0, slope]
    )

    widgets = ipw.HBox([Vbox1, Vbox2, Vbox3])
    graph = ipw.interactive_output(
        W3_interactiveplot_V,
        {
            "H0_1": H0_1,
            "H0_2": H0_2,
            "T1": T1,
            "T2": T2,
            "angle1": angle1,
            "angle2": angle2,
            "h0": h0,
            "slope_in": slope,
        },
    )

    display(widgets, graph)


# W3_show_V()

### U0

#### Coding

In [ ]:
def W3_plot_u0():
    # define the name of the function that the students will make
    function_name = "W3_u0"

    # define the correct function
    def correct_function(x_range, H0, T, h0, slope, angle):
        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(h0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        # page 199 of the book
        omega = 2 * np.pi / T
        u0 = 0.5 * omega * H / np.sinh(k * h)

        return u0

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.01 = 0.1%

    # set the size of the figure
    fig = Figure((5, 2.5))
    xlabel = "Cross-shore location (x) [m]"
    ylabel = "U0 [m/s]"

    # call the function that builds the backend.
    check_code_function_RANGE(
        fig, function_name, correct_function, f_margin, xlabel=xlabel, ylabel=ylabel
    )

#### Graph

In [ ]:
def W3_interactiveplot_U0(H0_1, H0_2, T1, T2, angle1, angle2, h0, slope_in):
    slope = 1.0 / slope_in  # bed slope [-]
    h0  # offshore water depth [m]
    x_max = round((h0 + 2) / slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    zbed = -h0 + slope * x  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(9, 4), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]")

    def calc(T, H0, angle, nr):
        # given:
        gamma = 0.8  # wave breaking ratio
        rho = 1025

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        # page 199 of the book
        omega = 2 * np.pi / T
        u0 = 0.5 * omega * H / np.sinh(k * h)

        axs[1].plot(x_water, H, label="H$_" + str(nr) + "$")
        axs[2].plot(x_water, u0, label="U0$_" + str(nr) + "$")

    # Calculate and plot the lines
    calc(T1, H0_1, angle1, nr=1)
    calc(T2, H0_2, angle2, nr=2)

    # adjust the layout
    for ax in axs:
        ax.legend(loc="lower left")
        ax.xaxis.set_visible(False)
        ax.legend(loc="best")
        ax.xaxis.set_visible(False)
    axs[2].xaxis.set_visible(True)

    # add descriptions
    axs[0].set_ylabel("Cross section")
    axs[1].set_ylabel("Wave height \n (H) [m]")
    axs[2].set_ylabel("U0 [m/s]")
    axs[2].set_xlabel("Cross-shore location (x) [m]")


def W3_show_U0():
    H0_1 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    H0_2 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T1 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")

    angle1 = ipw.FloatText(
        value=0, min=0, max=90, step=0.1, description="angle 1 [O]"
    )  # degree
    angle2 = ipw.FloatText(value=45, min=0, max=90, step=0.1, description="angle 2 [O]")
    slope = ipw.FloatText(
        value=100, min=0.1, max=250, step=0.1, description="slope 1:..."
    )
    h0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), H0_1, T1, angle1]
    )
    Vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), H0_2, T2, angle2]
    )
    Vbox3 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), h0, slope]
    )

    widgets = ipw.HBox([Vbox1, Vbox2, Vbox3])
    graph = ipw.interactive_output(
        W3_interactiveplot_U0,
        {
            "H0_1": H0_1,
            "H0_2": H0_2,
            "T1": T1,
            "T2": T2,
            "angle1": angle1,
            "angle2": angle2,
            "h0": h0,
            "slope_in": slope,
        },
    )

    display(widgets, graph)


# W3_show_U0()